In [1]:
import pandas as pd
from tables import *
import csv
import numpy as np
import pickle
import time

In [2]:
from tqdm import tqdm_notebook as tqdm

In [3]:
path = '/home/User1/data/self_citations/'

In [25]:
articles = {}
with open(path+'article.txt','r') as f:
    reader = csv.reader(f,delimiter='\t')
    next(reader)
    for line in reader:
        articles[int(line[0])] = int(line[2])

In [5]:
with open(path+'dict_citant.p','rb') as f:
    dict_citant = pickle.load(f)
with open(path+'dict_cite.p','rb') as f:
    dict_cite = pickle.load(f)

In [6]:
with open(path+'dict_cluster_ID.p','rb') as f:
    dict_cluster_ID = pickle.load(f)
with open(path+'dict_cluster_art.p','rb') as f:
    dict_cluster_art = pickle.load(f)

In [7]:
authors = list(dict_cluster_ID.keys())

/home/User1/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
nb_authors = len(authors)

In [9]:
class self_cits_year_level(IsDescription):
    Cluster_ID = UInt32Col()
    Citant_ID = UInt32Col()
    Cite_ID = UInt16Col()

In [30]:
h5file = open_file(path + 'self_citations_year_level.h5', mode="w", title="Self Citations")
group = h5file.create_group("/", 'counts')
table = h5file.create_table(group, 'self_cits', self_cits_year_level)
h5file.close()

In [31]:
h5file = open_file(path + 'self_citations_year_level.h5', mode="a", title="Self Citations")
table = h5file.root.counts.self_cits
row = table.row

In [13]:
citants = list(dict_citant.keys())
nb_citants = len(citants)
idx = np.arange(nb_citants)
np.random.shuffle(idx)

In [ ]:
start_time = time.time()
f = open(path+'progress.txt','w')
for i in range(nb_citants):
    ID_Art = citants[idx[i]]
    if ID_Art in dict_cluster_art:
        year_citant = articles[ID_Art]
        authors_art = dict_cluster_art[ID_Art]
        refs_art = set(dict_citant[ID_Art])
        for ID in authors_art:
            arts_author = set(dict_cluster_ID[ID])
            self_refs = refs_art.intersection(arts_author)
            for sr in self_refs:
                row['Cluster_ID'] = ID
                row['Citant_ID'] = ID_Art
                row['Cite_ID'] = sr
                row.append()
        if i % 1000 == 1 :
            elapsed_time = time.time() - start_time
            elapsed_time_h = np.round(elapsed_time/3600,2)
            time_per_art = elapsed_time/i
            time_left_h = np.round(time_per_art*(nb_citants-i)/3600,2)
            perc = np.round(i/nb_citants*100,3)
            f.write('Progress: {} articles, {}%, Time since start: {}, Time left: {}\n'.format(i,perc,elapsed_time_h,time_left_h))    
            f.flush()
        table.flush()

In [ ]:
f.close()
h5file.close()